In [1]:
%pip install pandas numpy scikit-learn nltk


Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import nltk
from nltk.corpus import stopwords
import string

nltk.download('stopwords')
stop_words = set(stopwords.words('french'))


[nltk_data] Downloading package stopwords to /home/batou/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
df = pd.read_csv('../data/training_SVM/donnees_SVM.tsv', sep='\t')
df.head()

,Pertinence,Question,Reponse
0,NON,"Bonjour à toutes et à tous, Je suis en train d...","Bonjour, Je suis entrain de poser env. 100m² d..."
1,NON,Bonjours a Tous Voilà mon projet : ma maison e...,"Si moi, 10 ans chez un fabricant De tous ceux ..."
2,NON,"Bonjour. Mes vacances au ski étant annulées, j...","Oui oui, c'est bien ce que je dis personne ne ..."
3,NON,"Bonjour, Je possède une vieille ferme en pierr...","Bonjour à tous, je suis frappé du même mal que..."
4,NON,"Bonjour, Quelle est la meilleur solution pour ...",pataya a écrit: Merci pour ta réponse. Le ccmi...


In [6]:
data = pd.read_csv('../data/training_SVM/donnees_SVM.tsv', delimiter='\t')
print(data.head())

  Pertinence                                           Question  \
0        NON  Bonjour à toutes et à tous, Je suis en train d...   
1        NON  Bonjours a Tous Voilà mon projet : ma maison e...   
2        NON  Bonjour. Mes vacances au ski étant annulées, j...   
3        NON  Bonjour, Je possède une vieille ferme en pierr...   
4        NON  Bonjour, Quelle est la meilleur solution pour ...   

                                             Reponse  
0  Bonjour, Je suis entrain de poser env. 100m² d...  
1  Si moi, 10 ans chez un fabricant De tous ceux ...  
2  Oui oui, c'est bien ce que je dis personne ne ...  
3  Bonjour à tous, je suis frappé du même mal que...  
4  pataya a écrit: Merci pour ta réponse. Le ccmi...  


On convertit nos colonnes en chaine de str car certains élements étaient des float

In [7]:
##seule façon que j'ai trouvé pour convertir en str en utilisant la méthode astype de pandas    
data['Question'] = data['Question'].astype(str)
data['Reponse'] = data['Reponse'].astype(str)

Il faut prétraiter notre texte afin d'avoir des vecteurs cohérents


In [8]:
import re
data['texte'] = data['Question'] + " " + data['Reponse'] ##concaténation de la question et de la réponse
def clean_text(texte):
    texte = texte.lower()
    texte = re.sub(r'\W', ' ', texte)
    texte = re.sub(r'\s+', ' ', texte)
    return texte

In [9]:
data['texte'] = data['texte'].apply(clean_text)  ## application du prétraitement 

Vectorisation de nos données

In [10]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data['texte'])
y = data['Pertinence']

Découpage en train, test, dev 

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [14]:
model = SVC() ## j'ai décidé de ne pas mettre de kernel, juste appeler naturellement SVC me donnait les meilleurs résultats dans ma matrice de confusion
model.fit(X_train, y_train)

SVC()

Matrice de confusion avec les mesures de précision

In [16]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print('Précision:', accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

         NON       0.45      0.19      0.26        27
         OUI       0.63      0.86      0.73        44

    accuracy                           0.61        71
   macro avg       0.54      0.52      0.50        71
weighted avg       0.57      0.61      0.55        71

Précision: 0.6056338028169014


Utilisation de notre modele SVM sur notre corpus afin d'évaluer la pertinence des réponses

In [19]:
new_data = pd.read_csv('../data/dataset/dataset_complet.tsv', delimiter='\t')

## conversion des colonnes en str car certains éléments étaient des floats
new_data['Question'] = new_data['Question'].astype(str)
new_data['Reponse'] = new_data['Reponse'].astype(str)

new_data['texte'] = new_data['Question'] + " " + new_data['Reponse'] ##concaténation question + réponse (même principe que la 1ere fois)
new_data['texte'] = new_data['texte'].apply(clean_text)
X_new = vectorizer.transform(new_data['texte']) ## on applique notre modele sur notre corpus
predictions = model.predict(X_new)

new_data['Pertinence_prediction'] = predictions

print(new_data[['texte', 'Pertinence_prediction']])

                                                texte Pertinence_prediction
0   bonjour à tous je suis arrivé sur ce forum en ...                   OUI
1   bonjour je possède une vieille ferme en pierre...                   OUI
2   bonjour je me permet de vous poster une petite...                   OUI
3   bonjour suite à une fuite dans la toiture reso...                   OUI
4   bonjour à tous je souhaite m équiper d un pack...                   OUI
5   bonjour je me permet de vous diffuser une vidé...                   OUI
6   bonjour à tous je souhaite rénover ma maison e...                   OUI
7   bonjour mes vacances au ski étant annulées je ...                   OUI
8   bonjour à tous alors voila avec ma compagne no...                   OUI
9   bonjour je refais ma cuisine et j ai trouvé un...                   OUI
10  jeremie25 l a demandé ici c est chose faite bi...                   OUI
11  bonjour à tous je viens ici chercher quelques ...                   OUI
12  bonjour 